In [1]:
#導入所需套件
from sklearn.externals import joblib 

#匯入模型
model1 = joblib.load('right_type.pkl')
model2 = joblib.load('left_type.pkl')

D:\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
#導入所需套件
import pandas as pd
import numpy as np
import os

path='Test_DataSet\\'
file_path=os.listdir(path)
right_list=list()
left_list=list()

#使用for迴圈設定完整的.xlsx路徑，讀進raw data後放入right_list及left_list
for i in file_path:
    if i!='test_main_table.xlsx':
        dat_path=path+i
        excel_path=os.listdir(dat_path)
        for j in excel_path:
            if j=='pressure.xlsx':
                A=pd.read_excel(dat_path+"\\"+j)
                A=A.drop(['Unnamed: 0','TimeStamp'], axis=1)                
                for k in A:
                    if k=='pressure_right':
                        right_list.append([A[k]])
                    elif k=='pressure_left':
                        left_list.append([A[k]])
print("right_list_shape:",str(np.array(right_list).shape))
print("left_list_shape:",str(np.array(left_list).shape))

right_list_shape: (40, 1)
left_list_shape: (40, 1)


In [3]:
#導入所需套件
from keras.preprocessing import sequence

#使用pad_sequences進行data補值，並轉換list維度
cnt=0
for i in right_list:
    right_list[cnt]=sequence.pad_sequences(i,dtype='float64',maxlen=7443)          
    cnt+=1
print("right_list_shape:",str(np.array(right_list).shape))                                  
right_list = np.reshape(right_list,(-1,7443))                           #三維轉二維
print("right_list_shape:",str(np.array(right_list).shape))              

cnt=0
for i in left_list:
    left_list[cnt]=sequence.pad_sequences(i,dtype='float64',maxlen=7443)          
    cnt+=1
print("left_list_shape:",str(np.array(left_list).shape))                                  
left_list = np.reshape(left_list,(-1,7443))                           #三維轉二維
print("left_list_shape:",str(np.array(left_list).shape))      

Using TensorFlow backend.


right_list_shape: (40, 1, 7443)
right_list_shape: (40, 7443)
left_list_shape: (40, 1, 7443)
left_list_shape: (40, 7443)


In [4]:
#導入所需套件
from sklearn import preprocessing

#對data進行正規化計算
right_list=right_list/right_list.max()
print("right_list[39]:",right_list[39])

left_list=left_list/left_list.max()
print("left_list[39]:",left_list[39])

right_list[39]: [0.         0.         0.         ... 0.00942127 0.00807537 0.00807537]
left_list[39]: [0.         0.         0.         ... 0.00846262 0.00987306 0.00987306]


In [5]:
#進行測試集預測
predict1=model1.predict(right_list)
predict2=model2.predict(left_list)
print("predict1:",predict1)
print("predict2:",predict2)

predict1: [3 3 4 3 4 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3]
predict2: [3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3]


In [6]:
#把預測結果印出並儲存在test_quality_type.xlsx
sub=pd.DataFrame(columns=['key','left_ng_s','left_normal_s','left_best','left_normal_l','left_ng_l','right_ng_s'\
                          ,'right_normal_s','right_best','right_normal_l','right_ng_l'])
key=list()
for i in file_path[0:40]:
    key.append(i[5:])
sub['key']=key

sub=sub.fillna(0)
cnt=0
for i in predict1:
    if i==3:
        sub['right_normal_l'][cnt]=1
    elif i==4:
        sub['right_ng_l'][cnt]=1
    cnt+=1
cnt=0
for i in predict2:
    if i==2:
        sub['left_best'][cnt]=1
    elif i==3:
        sub['left_normal_l'][cnt]=1
    cnt+=1
sub.to_excel('test_quality_type.xlsx')
sub

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning:

,key,left_ng_s,left_normal_s,left_best,left_normal_l,left_ng_l,right_ng_s,right_normal_s,right_best,right_normal_l,right_ng_l
0,20190709144841,0,0,0,1,0,0,0,0,1,0
1,20190709144911,0,0,0,1,0,0,0,0,1,0
2,20190709145610,0,0,0,1,0,0,0,0,0,1
3,20190709145848,0,0,0,1,0,0,0,0,1,0
4,20190709150235,0,0,0,1,0,0,0,0,0,1
5,20190709150259,0,0,0,1,0,0,0,0,1,0
6,20190709152333,0,0,0,1,0,0,0,0,0,1
7,20190709152358,0,0,0,1,0,0,0,0,1,0
8,20190709152858,0,0,0,1,0,0,0,0,1,0
9,20190709153213,0,0,0,1,0,0,0,0,1,0
